# Starbattle (星战)

> Rules 


The rules of Star Battle are simple:
You have to place stars on the grid according to the rules:
- 2 stars cannot be adjacent horizontally, vertically or diagonally.
- For 1★ puzzles, you have to place 1 star on each row, column and shape.
- For 2★ puzzles, the stars per row, column and shape must be 2, etc.

-------------

> 游戏规则

按如下要求在格子上放置星星：
- 任意两颗星星不能在横向、纵向或对角上相邻。
- 对于1★谜题，每行、每列及每个区块（黑色粗线条标识）上需放置1颗星星。
- 对于2★谜题, 每行、每列及每个区块（黑色粗线条标识）上需放置2颗星星，以此类推。

<https://www.puzzle-star-battle.com>

In [44]:
from ortools.sat.python import cp_model as cp

def StarbattleSolver(X, Y, grid, star):
    model = cp.CpModel()
    solver = cp.CpSolver()
    cells = dict()
    x = dict()
    aux_starbattle = {}
    for i in range(Y):
        for j in range(X):
            x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
            if grid[i * X + j] not in cells:
                cells[grid[i * X + j]] = []
            cells[grid[i * X + j]].append([i, j])
            aux_starbattle[i, j, "aux"] = model.NewBoolVar(f"x[{i}, {j}, aux]")
    
    for _, val in cells.items():
        model.Add(sum(x[i[0], i[1]] for i in val) == star)
    for i in range(Y):
        model.Add(sum(x[i, j] for j in range(X)) == star)
    for j in range(X):
        model.Add(sum(x[i, j] for i in range(Y)) == star)
    
    directions = [
        [-1,-1], [-1,0], [-1,1],
        [0, -1], [0, 0], [0, 1],
        [1, -1], [1, 0], [1, 1]
    ]
    for i in range(Y):
        for j in range(X):
            
            model.Add(x[i, j] == 1).OnlyEnforceIf(aux_starbattle[i, j, "aux"])
            model.Add(x[i, j] == 0).OnlyEnforceIf(aux_starbattle[i, j, "aux"].Not())
            
            temp_node = []
            for direct in directions:
                if i + direct[0] >= 0 and i + direct[0] < Y and \
                    j + direct[1] >= 0 and j + direct[1] < X:
                        temp_node.append(x[i + direct[0], j + direct[1]])
                        # print(f"{i + direct[0]} - {j + direct[1]}")
            model.Add(sum(temp_node) <= 1).OnlyEnforceIf(aux_starbattle[i, j, "aux"])
            # print(f"ADDD {len(temp_node)}")

    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        print("FOUND OPTIMAL")
        outputline = ""
        for idx in range(len(grid)):
            if solver.Value(x[idx // X, idx % X]) == 1:
                outputline += "# "
            else:
                outputline += ". "
            if idx % X == X - 1:
                print(outputline)
                outputline = ""



In [45]:
if __name__ == "__main__":
    grid = "AAAAAAABBBBBBCAADABBBBBBEBCCFADAAGGBEEEBBCFDDDAGGGEHEBBCFDDDFFIGGHHBBCFDFFFIIIGGHBHCFFFIIIJJGHHHHCKKFFFIJJGJJHHCKIIIIIIJJJJJCCKIKKKKIIJLLLCCKKKKKKKKJLLLLCMKMMKNNNJLLLCCMKKMKNLLLLLCCCMMMMNNNNNNLLCC"
    # grid = "ABBBBAACDDCCCDDEEEDDEEEDD"
    StarbattleSolver(X = 14,Y=  14,grid =  grid, star = 3)

FOUND OPTIMAL
. . # . . . # . . . . . # . 
# . . . # . . . . . # . . . 
. . # . . . # . # . . . . . 
# . . . . . . . . . # . # . 
. . . # . # . . # . . . . . 
. # . . . . . . . . # . # . 
. . . . # . # . # . . . . . 
. . # . . . . . . . . # . # 
. . . . # . . # . # . . . . 
. # . . . . . . . . . # . # 
. . . # . # . # . . . . . . 
# . . . . . . . . # . # . . 
. . . # . # . . . . . . . # 
. # . . . . . # . # . . . . 
196


![](../assets/figures/Starbattle1.png)

![](../assets/figures/Starbattle2.png)